### opencv haar을 이용해 crop 하기
- 얼굴이 검출되면 눈이 있는지 확인하고 crop
- 얼굴이 여러 개 검출되면 눈이 있는지 확인하고 가장 가능성 있는 부분 crop
- 눈의 개수가 2개가 아니면 잘못 찾았다고 판단하고 정해진 부분 crop
- 얼굴이 검출되지 않으면 정해진 부분 crop

## 크기 고정 240 x 350
- 스케일 고려 x
- 원본: 384 x 512 / crop: 240 x 350
- 가끔 이상한 것들이 있지만 최대한 잘되는걸로!

#### import

In [33]:
import os
import glob
from PIL import Image
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

#### face, eye haar

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
eye_casecade = cv2.CascadeClassifier('haarcascade_eye.xml')

#### 기타 함수

In [3]:
# 여러개의 얼굴 중 중심에 더 가까운 box 선택
def dist(center, mid): # center: 사진 중심, mid: box 중심 (x, y) 
    dist = ((center[0] - mid[0])**2 + (center[1] - mid[1])**2) ** (1/2)
    return dist

# 하나의 얼굴이 너무 벗어나지 않았는지 확인
def face(center, xywh): # center: 사진 중심
    if center[0] < xywh[0] or center[1] < xywh[1]:
        return False
    if center[0] > xywh[0]+xywh[2] or center[1] > xywh[1]+xywh[3]:
        return False
    return True

### Train

In [48]:
path = './train/train/images/'
crop_path = './train/train/crop_images2/'
person = os.listdir(path)

In [49]:
for j, p in tqdm(enumerate(person)): # 사람
    # print(p)
    # 사진 종류 .jpg / .jpeg
    images = glob.glob(os.path.join(path, p, '*.jpg'))
    images += glob.glob(os.path.join(path, p, '*.jpeg'))
    images += glob.glob(os.path.join(path, p, '*.png'))

    for i, image in enumerate(images): # 이미지
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.01,
                minNeighbors=5,
                minSize=(130, 130)
        )

        if len(faces) == 0: # 얼굴 X -> 정해진 사이즈 crop
            cropped = img[80:80+350, 60:60+240]
            
        elif len(faces) == 1: # 얼굴 하나 검출
            x, y, w, h = faces[0]
            if face((180, 255), (x, y, w, h)): # 범위 안에 있다면 crop
                mid_x, mid_y = max(120, x+w//2), max(175, y+h//2)
                cropped = img[mid_y-175:mid_y+175, mid_x-120:mid_x+120]
            else: # 범위 안에 없다면
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w]
                eyes = eye_casecade.detectMultiScale(roi_gray, minSize=(35, 35))
                if len(eyes) >= 2: # 그렇지만 눈 2개가 검출되면
                    mid_x, mid_y = max(120, x+w//2), max(175, y+h//2)
                    cropped = img[mid_y-175:mid_y+175, mid_x-120:mid_x+120]
                else: # 아니면 -> 정해진 사이즈 crop
                    cropped = img[80:80+350, 60:60+240]
                    
        else: # 얼굴 여러 개 검출
            n_eyes = 0
            center_dist = 500 # center에 가까운 얼굴 뽑기
            max_eyes_xywh = (0, 0, 0, 0)
            for (x, y, w, h) in faces:
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w]
                eyes = eye_casecade.detectMultiScale(roi_gray, minSize=(35, 35))
                c_dist = dist((180, 255), (x, y))
                if len(eyes) >= n_eyes and center_dist > c_dist: # 눈의 개수가 더 많으면서, 더 center에 가까운
                    n_eyes = len(eyes)
                    max_eyes_xywh = (x, y, w, h)
                    center_dist = c_dist
            if n_eyes < 2: # 눈이 2개 미만이면 -> 정해진 사이즈 crop
                cropped = img[80:80+350, 60:60+240]
            else: # 아니면 가장 유력한 box
                x, y, w, h = max_eyes_xywh
                mid_x, mid_y = max(120, x+w//2), max(175, y+h//2)
                cropped = img[mid_y-175:mid_y+175, mid_x-120:mid_x+120]
        
        # cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
        
        make_path = crop_path + p 
        if not os.path.isdir(make_path): 
            os.mkdir(make_path)
        save_path = os.path.join(make_path, image.split('\\')[-1])
        cv2.imwrite(save_path, cropped)
    time.sleep(0.1)

2700it [05:12,  8.64it/s]


#### 사람 수 확인

In [51]:
path = './train/train/images/'
crop_path = './train/train/crop_images2/'
person = os.listdir(path)
crop_person = os.listdir(crop_path)

In [45]:
len(person)

2700

In [52]:
len(crop_person)  # 2627

2700

In [47]:
for i, j  in zip(person, crop_person):
    if i!=j:
        print(i,j)
        break

006056_female_Asian_19 006075_male_Asian_18


#### 사진 다 7개인지 확인!

In [53]:
for j, p in tqdm(enumerate(crop_person)): # 사람
    # print(p)
    # 사진 종류 .jpg / .jpeg
    images = glob.glob(os.path.join(path, p, '*.jpg'))
    images += glob.glob(os.path.join(path, p, '*.jpeg'))
    images += glob.glob(os.path.join(path, p, '*.png'))
    if len(images)!=7:
        print(p)

2700it [00:00, 3534.03it/s]


### Eval

In [37]:
path = './train/eval/images/'
crop_path = './train/eval/crop_images/'

In [38]:
images = glob.glob(os.path.join(path, '*.jpg'))
images += glob.glob(os.path.join(path, '*.jpeg'))
# images += glob.glob(os.path.join(path, p, '*.png'))

for i, image in tqdm(enumerate(images)): # 이미지
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.01,
            minNeighbors=5,
            minSize=(130, 130)
    )

    if len(faces) == 0: # 얼굴 X -> 정해진 사이즈 crop
        cropped = img[80:80+350, 60:60+240]

    elif len(faces) == 1: # 얼굴 하나 검출
        x, y, w, h = faces[0]
        if face((180, 255), (x, y, w, h)): # 범위 안에 있다면 crop
            mid_x, mid_y = max(120, x+w//2), max(175, y+h//2)
            cropped = img[mid_y-175:mid_y+175, mid_x-120:mid_x+120]
        else: # 범위 안에 없다면
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_casecade.detectMultiScale(roi_gray, minSize=(35, 35))
            if len(eyes) >= 2: # 그렇지만 눈 2개가 검출되면
                mid_x, mid_y = max(120, x+w//2), max(175, y+h//2)
                cropped = img[mid_y-175:mid_y+175, mid_x-120:mid_x+120]
            else: # 아니면 -> 정해진 사이즈 crop
                cropped = img[80:80+350, 60:60+240]

    else: # 얼굴 여러 개 검출
        n_eyes = 0
        center_dist = 500 # center에 가까운 얼굴 뽑기
        max_eyes_xywh = (0, 0, 0, 0)
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_casecade.detectMultiScale(roi_gray, minSize=(35, 35))
            c_dist = dist((180, 255), (x, y))
            if len(eyes) >= n_eyes and center_dist > c_dist: # 눈의 개수가 더 많으면서, 더 center에 가까운
                n_eyes = len(eyes)
                max_eyes_xywh = (x, y, w, h)
                center_dist = c_dist
        if n_eyes < 2: # 눈이 2개 미만이면 -> 정해진 사이즈 crop
            cropped = img[80:80+350, 60:60+240]
        else: # 아니면 가장 유력한 box
            x, y, w, h = max_eyes_xywh
            mid_x, mid_y = max(120, x+w//2), max(175, y+h//2)
            cropped = img[mid_y-175:mid_y+175, mid_x-120:mid_x+120]

    # cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)

    save_path = os.path.join(crop_path, image.split('\\')[-1])
    cv2.imwrite(save_path, cropped)
    
    time.sleep(0.1)

12600it [28:29,  7.37it/s]
